# Feature Selection using Particle Swarm Optimization (PSO)

The search space for this problem corresponds to the power set of the set of the 54 attributes, which means
$2^{54}$ possibilities. Denote such space by $\mathcal{S}$. A local search algorithm can seek the desired subset of attributes without having
to pass through each of the elements in this colossus space.

Before submitting this problem to the PSO algorithm, it is necessary to define the way solutions (subsets) will
be represented and the function to rank the solutions, also known as the *fitness function*.

## Solutions representation

For this problem, a solution $i$ will be represented by a vector $x^i \in [0,1]^{54}$, in which each component $j$, $x^i_j$, means the probability of selecting the corresponding attribute $j$. Denote $[0,1]^{54}$ by $\mathcal{S_2}$. A threshold $0 \leq \theta \leq 1$ defines 
if the attribute was selected or not: $x^i_j \geq \theta$ means that the attribute $j$ is selected in the solution $i$,
as implemented by the simple function below. In this implementation, $\theta = 0.5$:

In [ ]:
def is_selected(xij):
    '''
    Determines if attribute j in solution i was selected
    based on the value on position j of vector x^i.
    '''
    return xij >= 0.5

## Fitness function

This function will rank each solution according to the objective of selecting the most appropriate attribute for this classification problem. Since the purpose is to determine the class of an instance based on the attribute values, the similarity measure of correlation will be applied here. Correlation between two random variables $X$ and $Y$ is given by the
equation:

$$
corr(X,Y) = \frac{cov(X,Y)}{\sigma_X\sigma_Y}
$$

where $cov(X,Y)$ is the covariance between $X$ and $Y$. The fitness function is defined as the mean of the unsigned correlations (in $[0,1]$) of each selected attribute with respect to the class. Given that the PSO algorithm used here seeks to minimize the fitness value, the fitness function $f:\mathcal{S_2}\to\mathbb{R}$ is given by the equation:

$$
f(x^i) = 1 - \left(\frac{\sum_{j, x^i_j = 1} |corr(x^i_j,cover\_type)|}{\sum_{j, x^i_j=1} 1}\right).
$$

## Implementation

First of all, it is necessary to load the dataset:

In [ ]:
import pandas as pd
# read the dataset
dataset = pd.read_csv("datasets/new_dataset_covertype.csv")
# preview
dataset.head()

Correlations with respect to the class are fixed values, and can be calculated using the following:

In [ ]:
# compute attribute-class correlations
class_correlations = dataset.corr(method="pearson")['cover_type']

Now, define the fitness function by:

In [ ]:
import numpy as np

def f(x, theta=0.5):
    '''
    Takes a vector in [0,1]^54 and compute its fitness value.
    '''
    selected_attrs = list(map(is_selected, x))
    if (any(selected_attrs)):
        sum_corr = sum([abs(class_correlations[i]) for i in np.arange(0,dataset.shape[1]-1) if selected_attrs[i]])
        count_attrs = sum(selected_attrs)
        return 1 - (sum_corr/count_attrs)
    else:
        return 1

The PSO implementation used here comes from the `pyswarm` library. An amount of 30 executions is performed:

In [ ]:
test = pd.Series([1,2]+[True])
test

In [ ]:
from pyswarm import pso
import itertools
# define variables's lower bound
lb = np.zeros(dataset.shape[1] - 1)
# define variables' upper bound
ub = np.ones(dataset.shape[1] - 1)
# number of trials
max_trials = 5#30
# swarm size
swarm_sizes = [25, 50, 100]
# maximum iterations
max_iterations = [50, 100, 200]
# dataframe to store results
results_columns = pd.Index(['swarm_size', 'max_iterations', 'fitness']).append(dataset.columns[:-1])
results = pd.DataFrame(columns=results_columns)
# running PSO for max_trials times combining parameters
for swarm_size, max_iterations in itertools.product(*[swarm_sizes, max_iterations]):
    print("swarm_size = " + str(swarm_size) + ", max_iterations = " + str(max_iterations))
    # execute PSO for the selected parameters
    for trial in range(1, max_trials + 1):
        # execute PSO
        xopt, fopt = pso(f, lb, ub, swarmsize=swarm_size, maxiter=max_iterations)
        # booleanize vector
        xopt = list(map(is_selected, xopt))
        # print info
        print("Trial " + str(trial) + ": " + str(fopt) + ", selected " + str(sum(xopt)))
        # append result
        results = results.append(pd.Series([swarm_size, max_iterations, fopt] + xopt, index=results_columns), ignore_index=True)
results

Now, check the solution with the least fitness values:

In [ ]:
# take a solution with minimum fitness
selected = results.loc[results['fitness'] == results['fitness'].min()]
selected

Finally, store the results in a CSV for further usage:

In [ ]:
# write the results in a csv
results.to_csv('results/pso_selected_attributes.csv', index=False)